In [ ]:
import env
import gymnasium as gym 
import pandas as pd
import os

In [ ]:
# load training data - 1-min sampling rate
file_path = os.path.abspath('')
parent_path = os.path.dirname(file_path)
data_path = os.path.join(parent_path, 'data/disturbance_1min.csv')
data = pd.read_csv(data_path, index_col=[0])
index = range(0, len(data)*60, 60)
data.index = index

# sample every hour
dt = 3600
data = data.groupby([data.index // dt]).mean()
n = len(data)

In [ ]:
data.head()


In [ ]:
disturbance_names = ['out_temp', 'qint_lump', 'qwin_lump', 'qradin_lump']
disturbance = data[disturbance_names].values
disturbance.shape


In [ ]:
# test rc state space
rc_params = [10383.181640625, 499116.65625, 1321286.5, 1.53524649143219, 0.5000227689743042, 1.0003612041473389, 20.09742546081543]
x0 = [20, 20, 20]
x_high = [40, 80, 40]
x_low = [10, 10, 10]
n_actions = 101
u_high = [0]
u_low = [-20.0]

t_d = [t*dt for t in range(n)]


In [ ]:
from scipy import interpolate 
interp = interpolate.interp1d(t_d, disturbance, axis=0, kind='linear', fill_value='extrapolate') # 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'next'
interp(0)

In [ ]:

env = gym.make("R4C3Discrete-v0",
    rc_params = rc_params,
    x0 = x0,
    x_high = x_high,
    x_low = x_low,
    n_actions = n_actions,
    u_high = u_high,
    u_low = u_low,
    disturbances = (t_d, disturbance),
    ts = 0,
    te = 24.*dt,
    dt = dt).env


#obs, _ = gym.reset()